In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle(r"shared/Motor_Vehicle_Collisions_-_Crashes.pkl")

In [3]:
df.head() # getting an insight of the data

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN


In [4]:
crashes_by_borough = df['BOROUGH'].value_counts(dropna=True) # finding out borough wise crashes and ignoring the null values

In [5]:
crashes_by_borough_sorted = crashes_by_borough.sort_values(ascending=False) # sorting the crashes by borough in descending order

In [6]:
second_highest_borough = crashes_by_borough_sorted.index[1]  # extracting the second highest borough with crashes
second_highest_crashes = crashes_by_borough_sorted.iloc[1]

In [7]:
print(f"The borough with the second highest number of crashes is {second_highest_borough}, with {second_highest_crashes} crashes.")

The borough with the second highest number of crashes is QUEENS, with 372457 crashes.


In [8]:
populations = {
    'BRONX': 1446788,
    'BROOKLYN': 2648452,
    'MANHATTAN': 1638281,
    'QUEENS': 2330295,
    'STATEN ISLAND': 487155
}

In [9]:
crashes_per_100k = {
    borough: (crashes_by_borough.get(borough, 0) / populations[borough]) * 100000
    for borough in populations
} # calculating the number of crashes per 100000 people

In [11]:
max_crashes_borough = max(crashes_per_100k, key=crashes_per_100k.get)
max_crashes_rate = crashes_per_100k[max_crashes_borough]

In [12]:
print(f"The borough with the most crashes for every 100,000 people is {max_crashes_borough}, with a rate of {max_crashes_rate:.2f}.")# showing the data with exactly 2 decimal places

The borough with the most crashes for every 100,000 people is MANHATTAN, with a rate of 19121.63.


In [13]:
specified_factors_df = df[df['CONTRIBUTING FACTOR VEHICLE 1'] != 'Unspecified']

In [14]:
leading_cause_series = specified_factors_df['CONTRIBUTING FACTOR VEHICLE 1'].value_counts() #a series of each unique value in that column is been counted
leading_cause = leading_cause_series.idxmax() # To calculate the leading contributing factor for vehicle collision and the count of attributes related to that collision
leading_cause_count = leading_cause_series.max() 

In [15]:
total_specified_accidents = specified_factors_df['CONTRIBUTING FACTOR VEHICLE 1'].count() #total number of contributuing factors 

In [16]:
proportion_of_accidents = leading_cause_count / total_specified_accidents

In [17]:
print(f"The leading cause of crashes is '{leading_cause}' with {leading_cause_count} crashes,")
print(f"which represents {proportion_of_accidents:.2%} of the accidents with a specified contributing factor.")

The leading cause of crashes is 'Driver Inattention/Distraction' with 401262 crashes,
which represents 30.42% of the accidents with a specified contributing factor.


In [18]:
top_5_causes = leading_cause_series.head(5)

In [19]:
top_5_crashes_total = top_5_causes.sum()

In [20]:
total_specified_crashes = specified_factors_df.shape[0]
top_5_proportion = top_5_crashes_total / total_specified_crashes

# the top 5 causes and their proportion of total specified crashes
print("Top 5 causes of crashes:")
print(top_5_causes)
print(f"\nThe top 5 causes account for {top_5_proportion:.2%} of all specified crashes.")

Top 5 causes of crashes:
CONTRIBUTING FACTOR VEHICLE 1
Driver Inattention/Distraction    401262
Failure to Yield Right-of-Way     119166
Following Too Closely             107467
Backing Unsafely                   75042
Other Vehicular                    62688
Name: count, dtype: int64

The top 5 causes account for 57.76% of all specified crashes.


In [21]:
# Replacing missing values in 'NUMBER OF PERSONS KILLED' column with 0
df['NUMBER OF PERSONS KILLED'] = df['NUMBER OF PERSONS KILLED'].fillna(0)


In [22]:
# Filtering for accidents with two or more fatalities
accidents_with_two_or_more_fatalities = df[df['NUMBER OF PERSONS KILLED'] >= 2]

# total count of such accidents
total_count = accidents_with_two_or_more_fatalities.shape[0]

total_count

88

In [23]:
#the average number of accidents causing at least one death per 1000 accidents
accidents_with_deaths = df[df['NUMBER OF PERSONS KILLED'] >= 1].shape[0]
total_accidents = df.shape[0]

# Calculating the ratio and multiplying it by 1000 to find the average per 1000 accidents
average_deaths_per_1000_accidents = (accidents_with_deaths / total_accidents) * 1000

average_deaths_per_1000_accidents

1.3893258747079764

In [24]:
# the proportion of accidents without a Cross Street Name
proportion_no_cross_street = (df['CROSS STREET NAME'].isnull().sum() / len(df)) * 100

proportion_no_cross_street

37.4350983156158

In [25]:
# Counting the most common combinations of the two columns namely 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2'
vehicle_combinations = df.groupby(['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2']).size().reset_index(name='counts')

# Identifying the combination with the most accidents that happened
most_common_combination = vehicle_combinations.sort_values(by='counts', ascending=False).head(1)

most_common_combination

,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,counts
3948,Sedan,Sedan,197944


In [26]:
# Using the given assumptions that 'Alcohol Involvement' is a contributing factor and using the 'NUMBER OF PERSONS KILLED' field
# Filtering the dataset which consist of 'Alcohol Involvement' crashes
alcohol_involved_crashes = df[df['CONTRIBUTING FACTOR VEHICLE 1'] == 'Alcohol Involvement']

# Calculating the number of crashes  that resulted in at least one fatality
fatal_alcohol_crashes = alcohol_involved_crashes[alcohol_involved_crashes['NUMBER OF PERSONS KILLED'] > 0].shape[0]

# the total number of alcohol-involved crashes
total_alcohol_crashes = alcohol_involved_crashes.shape[0]

# the proportion of alcohol-involved crashes which resulted in fatalities
proportion_fatal_alcohol_crashes = fatal_alcohol_crashes / total_alcohol_crashes if total_alcohol_crashes > 0 else 0

fatal_alcohol_crashes, total_alcohol_crashes, proportion_fatal_alcohol_crashes

(101, 21656, 0.0046638345031400075)

In [27]:
print(df.columns)
#finding the names of the columns

Index(['CRASH DATE_CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE',
       'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME',
       'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
       'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',
       'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2',
       'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4',
       'CONTRIBUTING FACTOR VEHICLE 5', 'COLLISION_ID', 'VEHICLE TYPE CODE 1',
       'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4',
       'VEHICLE TYPE CODE 5'],
      dtype='object')


In [28]:
df['CRASH DATE_CRASH TIME'] = pd.to_datetime(df['CRASH DATE_CRASH TIME'])

# calculating the crashes occurring between 4 PM (16:00) and before 7 PM (19:00)
evening_rush_hour_crashes = df[(df['CRASH DATE_CRASH TIME'].dt.hour >= 16) & (df['CRASH DATE_CRASH TIME'].dt.hour < 19)]

# Calculate the proportion
total_crashes = len(df)
evening_rush_hour_crashes_count = len(evening_rush_hour_crashes)
proportion = evening_rush_hour_crashes_count / total_crashes * 100

proportion

20.514010935243242

In [29]:
# Filtering the dataset for crashes involving motorcycles
motorcycle_crashes = df[(df['VEHICLE TYPE CODE 1'].str.contains('MOTORCYCLE', na=False)) | 
                         (df['VEHICLE TYPE CODE 2'].str.contains('MOTORCYCLE', na=False))]

# Finding out the cases where injuries are greater than 0 and fatalities are equal to 0
injuries_no_fatalities = motorcycle_crashes[(motorcycle_crashes['NUMBER OF PERSONS INJURED'] > 0) & 
                                            (motorcycle_crashes['NUMBER OF PERSONS KILLED'] == 0)]

# Calculate the proportion
total_motorcycle_crashes = len(motorcycle_crashes)
injuries_no_fatalities_count = len(injuries_no_fatalities)
proportion = injuries_no_fatalities_count / total_motorcycle_crashes * 100

proportion

50.04565018912221

In [30]:
# Count the number of crashes where vehicle type is 'BICYCLE'
bicycle_crashes = df[(df['VEHICLE TYPE CODE 1'].str.contains('BICYCLE', na=False)) | 
                     (df['VEHICLE TYPE CODE 2'].str.contains('BICYCLE', na=False))]

# Getting the total count of crashes which involves bicycles
total_bicycle_crashes = len(bicycle_crashes)

total_bicycle_crashes

19108